In [15]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropoutpout
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

In [16]:

with open('dataset_pozytywizm.txt','r',encoding='utf-8') as txt:
    text = txt.read()
print('corpus length:', len(text))

corpus length: 112829


In [17]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 111


In [18]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 37597


In [19]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [20]:
y.shape

(37597, 111)

In [25]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(Dense(128, input_shape=(maxlen, len(chars))))
model.add(LSTM(128,return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(128))
model.add(LSTM(128))
model.add(Dense(len(chars), activation='softmax'))


Build model...


In [26]:
# # build the model: a single LSTM
# print('Build model...')
# model = Sequential()
# # model.add(LSTM(128, input_shape=(maxlen, len(chars))))
# model.add(Dense(len(chars), activation='softmax', input_shape=(maxlen, len(chars))))
# model.add(LSTM(len(chars)))
# model.add(Dense(len(chars), activation='softmax'))


In [27]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [28]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=20,
          callbacks=[print_callback])

Epoch 1/20
37504/37597 [============================>.] - ETA: 0s - loss: 3.5751
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "
Zachwytem serce moje,
Nie dla mnie już
"

Zachwytem serce moje,
Nie dla mnie już
 c c k k k łcrrc cc k kk   c k kcccc ckc   ccck kkc k  cck     cck c kk k  kc    c c k  kcckc   ccckc ckccc c k  k  kkckcccc  c  c k ckcca c cc kkc  c k c    ccccłce cc cc  c cc kkkc   ckc  ck kckcc c c ccc c   ckck cc c kcc c c kc ckcckcccc cc    ckc   k  k cc ckkc  k kcc  ckccc ckc ckcc    ck kck r rc ck kc cckk c  k  cckc  kkcy  kccc  ccc  ckccck kc cac ckkkc  kk   cc zzkc  kc    cckc cccc  rc 
----- diversity: 0.5
----- Generating with seed: "
Zachwytem serce moje,
Nie dla mnie już
"

Zachwytem serce moje,
Nie dla mnie już
r
c k cd kdcca u  c
 z k rccscaa  e ł t kkn c  crck ccekcdc aókn cc cd    clc c ói tkkeckcł  Keaccok nkccrcampc ckkccrscga c mktkcrkrłc ococe
ouc cmkprk  kc  nk t ci ckcaamrc ctue r zn  krz 
cccoyaakckccikec   co ceac

Sanctusó załae wzadymoryi, po wrónie, Takiydze zrybo uwesni,
Rrzocana,
Bię wiemrany, bab!

II Mel daszza kzy wodzołka dogrzyje, pocół gródhaszel mosu,
Ożełzy zrzalewie
       Zenacz ulo ckielnie, poz — dosprowieł,
Odzalykan sąrodzelkąch,




G nem śśtgapookyczagrz lenem pażna, kieweła.
Kiłla kochema ziem narooi,.

Nosne,




ToniegrzyOse
At lada dąwyzu,


Tapęma,
I bogośne
dum małnkou sieka zocu,
Ażcz —
----- diversity: 1.2
----- Generating with seed: "iłości obdziel wszystkich nas!



Sanctu"
iłości obdziel wszystkich nas!



Sanctuwym zyskzuwut
Coraki soęz irone
Mazaćkeczeioy sza łtę, swomike
Bookajmwó gu.
Te, chyzeeń setko śfutwy jem drzeńs jeFs — Loli,
Ragi nasty,
W nęte,
Ku szózom,
Rógziaś śichsiedy daśeg brozsne poskię,
Led pe sizznłę gocieva”stnie ne narlamche, idląl pocza,
Boz śbocęsnięwę.
Ady śnaruwkoch wopa, oodzykoronośeckowyt ośczzązsticenałzece!




Czu tząstech,
Astlnarnot duł dadzę,
Lyta, pukiwwatónicze sołotmó
37597/37597 [==============================] - 87s 2ms/step 

Gdy więc bo wieczy szarnie szer się się się wie szarysta słoszeczaszy,
Aczy ta nie posię się się to tor to szery stoczy pospoczaczyczych posiesza sieszesty w sid szalie słoszeczy staje szalie,
Na to stoszeczy ta szaresze to szaczy w szarzych,
I szarysty szer toszaczyczyszy niecza wie szarzesze to stosteczyczyszy toczy się swoty się toczy wie szar.
















I to się to posię się to to ty szaryszy proch
----- diversity: 0.5
----- Generating with seed: "uty:
Mokre nogi, mokre buty,
Gdy więc bo"
uty:
Mokre nogi, mokre buty,
Gdy więc boba w czie spażaczy oczo miech,
I mas lasze narnie to poprad na szumej narki,
A te,
Aak namy tomie,
Bo zia niernie się te przydwie,
Nie poczy nieczy stybie,
Boczu to ta to pocieky wyrla sthe z wie te.

Na soryry się się toch szerar chała,
Aczyw powie seme prowie słoszna czyszety słodzieba,
Na sięry swoskę ta rzare wia słoczyczyśnie,
A panieczymą morraryszyczyczyje się ty i ślie raszaczymyszy prochu
----- diversity: 1.0
----- Generating with seed: "uty:


Czy nadnaj słe siens mi gojiainajn praskty.
Por ga w sskie
Sotę siad ziu śtogur wa są cza i płatys i rudrzódnkle,
Alre grón szegkojlzagnie pilałylor giułią.
Młómda pykrze,
Tzy kajne mydaojęczki z fiesy nadeżna idzi,
I podę Miagu luśsię chewika cha zilni coty rręszezch
----- diversity: 1.2
----- Generating with seed: ",
Jak nad mleczkiem kotki mruczą,
Jak do"
,
Jak nad mleczkiem kotki mruczą,
Jak do rlo nujniko mol dudy Dhegzi sorobica…

Perzynwa — fywnie giukki,
Herdrzy plevsłsa lymoi kiesf stałi!
Jo gią,
Dnie jenłamiekąksa
Bze waskumdina,
Onyczkę rylilk puskam na jaz
W zokiąm psiłykst doję tmój swati go uk złyjni.
Starkiąw meninł

No rodusuł da z pa gra jtyżęcie tarny,
Oiconk zakamaćs,
Chzie, łasbąj,

Nwosztewe.
Bią dzorłe płowogu niaką naid glytą Jłrzu pkied lirloj.


Momą babuż!


Bzi pa
37597/37597 [==============================] - 95s 3ms/step - loss: 2.7038
Epoch 12/20
37504/37597 [============================>.] - ETA: 0s - loss: 2.7431
----- Generating text after Epoch: 11
---

I się szostody niesze stody się szoszosze,
I szostoko szody się posze się bosze niestarech się strzeche,
I stostoszo stosze nieste szo niesze szrocze,
I nie sie dosko szo szo szodzie się szodzie nare,
I swostodzie się się sto się się szosko szrukie,
I nie się szoskocze w szostokie się szrosze,
Cze pomie poskostosze szoszaszalarem posze,
A nieski
----- diversity: 0.5
----- Generating with seed: "ci w dnia blaski
Po swobodę, dla gniazda"
ci w dnia blaski
Po swobodę, dla gniazdam nie powieżoszaszrze szonokie,
Czym soszrejie nodzie szudziec swoczoche, się.


Areszona nieko zieżo się stoko,
Tzo miesy,
I biecie,
Turecze wied niecze todzie miem nieśska stustoszy



W w postowiem zowo spod grosare szyw kie lorecie.


Madzam zietniem kych zelosze wiezieszkie,
Wie paszrad stuskie zóry okie,
A nie kie starawie gokocza, naszen zieniech szie ziem dlate szeże,
Czą basto nesko,
I po
----- diversity: 1.0
----- Generating with seed: "ci w dnia blaski
Po swobodę, dla gniazda"
ci w dnia blaski
Po swobodę

Tera nociowier peźwa,
Zarew i wiekocieś ciem jestkzeczie o i liedzyta jalia… Thrzypknnych: poć…
Biej z bory ol ś wo rzenam


Lych pułozskę mrze stu o myłe nięśśwaną kwie, tułowy słowi piesąć krad znieśm udy pływiens
----- diversity: 1.2
----- Generating with seed: "okoły!
Rozwińcie skrzydlate hufce,
Gdzie"
okoły!
Rozwińcie skrzydlate hufce,
Gdziepiarzecez i tittchm: lejczie
Nan drombu ściebnwe.
Gosza!

Cyn swoć,
Się śi żom ttieśskiśkiejni śthe,
Zo bium.…



Weśszep wdy nonoli,
 „Sasktlde siyziaką mialiemabniemu,
Z są swiękysewośnkkie,
Cycisthża ciężchne bolszecyte dwięczywiśłi,
Amdzicz ziei jad tichtrai ocziel!….
Tzłatskłzyż degicinieczaię nache,
Gego łosna pówamlyjużewya pdrogorni,
 Trychśżagdu brzy zikrobloć
Głochdekackie jurebpi bumngw
37597/37597 [==============================] - 90s 2ms/step - loss: 2.7421
Epoch 19/20
37504/37597 [============================>.] - ETA: 0s - loss: 2.7272
----- Generating text after Epoch: 18
----- diversity: 0.2
----- Generating with seed: "rzucaj

In [29]:

neural_texts = list()
fan_fic_len = 400

for n in range(10):
    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.7]:

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence

        for i in range(fan_fic_len):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    neural_texts.append(generated)

In [30]:
neural_texts

['m!\n\n\n\nBijatyka\n\nPo wieczerzy, do łóżeczeje\nW tyte mszy,\nI kumwe, deż słzyła…\n\n\n\n\n\n\n\n\n\n\n\n  Bzy ieszcie przyjejc mmowiej.\n\n   Czywna pam łiem soczo!\n\n\n\n\n\n\n\n\n\nS  Noszek ty zstarlo tumemca żyba… Sło śdaky…\nI gokieny;\n\nII świegieciej tukaje osta duli go sy śi,\nI tęm niews chłak potadz taje swadn stu\nJek jaż,\n\n\n\nŻo — biem bra ustochni\nW paże szacie ram dydni,\nGody chytna szięcecy kokłu parumiebasły,\nZ stelą na, sano wodrzym woro tał ni niery;',
 'Bo za takie awantury,\n  Drą ze skóry!\n\n\n\n\n\n\n\n\n\nPtoty słowiema,\nMy nace,\nGakdzieł nażaska…\nI potwachy ka mołani!\nGażdożać do wonko człarana ocznem\nI tlo ożaczcace rod tam matyciesna słzymce\n\n\nOt szawe nie test zytecieci prurę słunie.\nMo iše pal posie w nie.\nI bożie!\nPadzoź ninie,\nZsarnnora udźgliestew,\nSz frzyryść\nWaże taier sie ju te wna poniena, zpodźni.\n\nII tam tendź się dhylkie boszycz nanem zażawospy…\nI wnie szłakoch,\nGie że nież\nG ',
 'ws.\nWith every secret now reveale

In [31]:
model.save('.\model3.h5')
model.save_weights('.\model3Weights.h5')